# Capstone Project — The Battle of Neighborhoods- The London Insight

### Setting up a shopping mall in London Neigbourhood, United Kingdom.

## 1 Introduction

London is the capital and largest city of England and the United Kingdom.There really isn’t any shopping experience quite like London. London offers several distinct shopping areas, streets, outlets and malls The population of London has grown considerably over the last decades and for many visiting, shopping mall is a perfect way to relax and enjoy themselves. This present a good businness opporturnity for a property development company that wants to invest on opening new shopping mall.

## Business Problem

The purpose of this capstone project is to help prospective property developers explore the best locations arround london Boroughs. it will help them make efficient decision on selecting best neighborhood to open new shopping mall.

## Methodology

For this project we are going to use the K-means Clustering algorithm to cluster the neighborhoods and focused on geospatial analysis of the London to understand which would be the best neighborhoods to open a new mall. Foursquare API to provide different venue categories. The business question is: In London, if a property developer is looking to open a new shopping mall, where would you recommend that they open it?

## Data Section

Data Link:https://en.wikipedia.org/wiki/List_of_places_in_London

#### we import the necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

!pip install folium
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


#### Download the wikppedia page to extract the necessary informations

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_places_in_London'

In [3]:
tables = pd.read_html(url)

In [4]:
len(tables)

3

In [5]:
tables[0]

,Unnamed: 0,London borough,Inner/Outer,Sub-region[1],London Assembly[2]
0,NaN,Barking and Dagenham,Outer,North East,City and East
1,NaN,Barnet,Outer,North,Barnet and Camden
2,NaN,Bexley,Outer,South East,Bexley and Bromley
3,NaN,Brent,Outer,West,Brent and Harrow
4,NaN,Bromley,Outer,South East,Bexley and Bromley
5,NaN,Camden,Inner,North,Barnet and Camden
6,NaN,Croydon,Outer,South West,Croydon and Sutton
7,NaN,Ealing,Outer,West,Ealing and Hillingdon
8,NaN,Enfield,Outer,North,Enfield and Haringey
9,NaN,Greenwich,Inner †,South East,Greenwich and Lewisham


In [6]:
df_table = tables[0]

Drop the unnecessary columns

In [7]:
df_table = df_table.drop(['Unnamed: 0','Inner/Outer','Sub-region[1]','London Assembly[2]'], axis=1)

In [8]:
df_table

,London borough
0,Barking and Dagenham
1,Barnet
2,Bexley
3,Brent
4,Bromley
5,Camden
6,Croydon
7,Ealing
8,Enfield
9,Greenwich


Save the dataframe to a new csv file

In [9]:
df_table.to_csv('df.csv', index=False, header=False)

In [10]:
header = ['BoroughName']

In [11]:
df = pd.read_csv('df.csv', names=header)

In [12]:
df.head()

,BoroughName
0,Barking and Dagenham
1,Barnet
2,Bexley
3,Brent
4,Bromley


In [13]:
df.shape

(32, 1)

### Get the geo-coordinates of London city using the geopy library of python

In [14]:
address = 'London, UK'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


#### So we proceed to applying it to our dataframe df

In [15]:
latitude = []
longitude = []
Borough = df['BoroughName']
for i in range(len(Borough)):
    address = str(Borough[i] + ', England')
    geolocator = Nominatim(user_agent="london_explorer")
    #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    location = geolocator.geocode(address)
    try: 
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append("Not Found")
        longitude.append("Not Found")
    
print('Done!')

Done!


#### Then we proceed to store the location data — latitude and longitude as follows.

In [16]:
df_coord = df
df_coord['Latitude'] = latitude
df_coord['Longitude'] = longitude
df_coord.head()

,BoroughName,Latitude,Longitude
0,Barking and Dagenham,51.554117,0.150504
1,Barnet,51.653090,-0.200226
2,Bexley,51.441679,0.150488
3,Brent,51.442026,0.231523
4,Bromley,51.402805,0.014814


### Create a map of London city with all the boroughs superimposed on it.

In [17]:
# create map of New York using latitude and longitude values
map_london = folium.Map(location=[51.5073219, -0.1276474], zoom_start=10)

# add markers to map
for lat, lng, boroughName in zip(df_coord['Latitude'], df_coord['Longitude'], df_coord['BoroughName']):
    label = '{}'.format(boroughName)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### let's simplify the above map and segment and cluster only the neighborhoods in Newham and create a new dataframe of the new_neig.

In [18]:
Borough = ['Newham','Newham','Newham','Newham',
          'Newham','Newham','Newham','Newham',
          'Newham','Newham','Newham','Newham',
          'Newham','Newham','Newham']



Neighborhood = ['Beckton','Canning Town','Cyprus','East Ham','East Village','Forest Gate',
'Manor Park','Maryland','North Woolwich','Plashet',
'Silvertown', 'Stratford','Upton Park','Wallend','West Ham']


Latitude = ['','','','','','','','','','','','','','','']
Longitude = ['','','','','','','','','','','','','','','']

df_neigh = {'Neighborhood': Neighborhood,'Borough':Borough,'Latitude': Latitude,'Longitude':Longitude}
new_neig = pd.DataFrame(data=df_neigh, columns=['Neighborhood', 'Borough', 'Latitude', 'Longitude'], index=None)

new_neig

,Neighborhood,Borough,Latitude,Longitude
0,Beckton,Newham,,
1,Canning Town,Newham,,
2,Cyprus,Newham,,
3,East Ham,Newham,,
4,East Village,Newham,,
5,Forest Gate,Newham,,
6,Manor Park,Newham,,
7,Maryland,Newham,,
8,North Woolwich,Newham,,
9,Plashet,Newham,,


#### Get the geo-coordinates of Newham Borough using the geopy library

In [19]:
address = 'Newham, London'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Newham are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Newham are 51.52999955, 0.02931796029382208.


#### we proceed to applying it to our dataframe df_coord

In [20]:
latitude = []
longitude = []
Borough = new_neig['Borough']
Neighborhood = new_neig['Neighborhood']
for i in range(len(Neighborhood)):
    address = str(Neighborhood[i] + ', England')
    geolocator = Nominatim(user_agent="explorer")
    #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    location = geolocator.geocode(address)
    try: 
        latitude.append(location.latitude)
        longitude.append(location.longitude)
    except:
        latitude.append("Not Found")
        longitude.append("Not Found")
    
print('Done!')

Done!


#### Then we proceed to store the location data — latitude and longitude as follows.

In [21]:
df_coord = new_neig
df_coord['Latitude'] = latitude
df_coord['Longitude'] = longitude
df_coord.head(14)

,Neighborhood,Borough,Latitude,Longitude
0,Beckton,Newham,51.516080,0.059426
1,Canning Town,Newham,51.513989,0.008299
2,Cyprus,Newham,51.508478,0.063969
3,East Ham,Newham,51.532963,0.055320
4,East Village,Newham,51.548108,-0.009177
5,Forest Gate,Newham,51.549524,0.024925
6,Manor Park,Newham,51.426949,-0.135507
7,Maryland,Newham,51.546053,0.005922
8,North Woolwich,Newham,51.500407,0.064154
9,Plashet,Newham,51.540008,0.039274


Let's get the geographical coordinates of Newham

In [22]:
locations = df_coord[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[51.5460532, 0.0059223]

As we did with all of London, let's visualize the map of Newham and their neighborhoods using folium

In [23]:
map = folium.Map(location=[51.5460532, 0.0059223], zoom_start=11)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_coord['Neighborhood'][point]).add_to(map)
map

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

To make calls to the Foursquare API, you need to register and get your credentials to use here as client id and client secret.


In [34]:
# The code was removed by Watson Studio for sharing.

### We can now explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [35]:
df_coord.loc[0, 'Neighborhood']

'Beckton'

Get the neighborhood's latitude and longitude values

In [36]:
neighborhood_latitude = df_coord.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_coord.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_coord.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beckton are 51.5160797, 0.0594257.


#### Now, let's get the top 100 venues that are in Beckton within a radius of 500 meters

First, let's create the GET request URL. Name your URL url

In [37]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format("VR2R4MBCL2535OVY5FBQIFRKOZEG1DSOT5YMD5VT23Q4VZTO", "VXGRGIDRWNUANNGMCLXLW23Z454DHPHBYXJD4POBG3VMZBTM", 51.5160797, 0.0594257, "20180605", 500, 100)
url

'https://api.foursquare.com/v2/venues/explore?client_id=VR2R4MBCL2535OVY5FBQIFRKOZEG1DSOT5YMD5VT23Q4VZTO&client_secret=VXGRGIDRWNUANNGMCLXLW23Z454DHPHBYXJD4POBG3VMZBTM&ll=51.5160797,0.0594257&v=20180605&radius=500&limit=100'

Let's Send the GET request and examine the resutls

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fff193c1c80e062c55ae577'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Beckton',
  'headerFullLocation': 'Beckton, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 14,
  'suggestedBounds': {'ne': {'lat': 51.5205797045, 'lng': 0.06664350286675196},
   'sw': {'lat': 51.5115796955, 'lng': 0.05220789713324805}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e0c524018a89ad010688d2b',
       'name': 'East london Gymnastics Club',
       'location': {'lat': 51.514106774737556,
        'lng': 0.060155068624099396,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.514106774737556,
          'lng': 0.0601

Lets define the get_category_type function

In [39]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,East london Gymnastics Club,Gym / Fitness Center,51.514107,0.060155
1,Home Bargains,Discount Store,51.516790,0.062967
2,Lidl,Supermarket,51.515982,0.054794
3,Lituanica,Grocery Store,51.516442,0.062927
4,Premier Inn London Beckton,Hotel,51.515151,0.061013


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

14 venues were returned by Foursquare.


## 2 Explore Neighborhoods in Newham

### Now we create a function to repeat the same process to all the neighborhoods in Manhattan

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called newham_venues.

In [43]:
newham_venues = getNearbyVenues(names=df_coord['Neighborhood'],
                                   latitudes=df_coord['Latitude'],
                                   longitudes=df_coord['Longitude']
                                  )

Beckton
Canning Town
Cyprus
East Ham
East Village
Forest Gate
Manor Park
Maryland
North Woolwich
Plashet
Silvertown
Stratford
Upton Park
Wallend
West Ham


### Let's check the size of the resulting dataframe

In [44]:
print(newham_venues.shape)
newham_venues.head()

(248, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beckton,51.51608,0.059426,East london Gymnastics Club,51.514107,0.060155,Gym / Fitness Center
1,Beckton,51.51608,0.059426,Home Bargains,51.516790,0.062967,Discount Store
2,Beckton,51.51608,0.059426,Lidl,51.515982,0.054794,Supermarket
3,Beckton,51.51608,0.059426,Lituanica,51.516442,0.062927,Grocery Store
4,Beckton,51.51608,0.059426,Premier Inn London Beckton,51.515151,0.061013,Hotel


### Let's check how many venues were returned for each neighborhood

In [45]:
newham_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beckton,14,14,14,14,14,14
Canning Town,28,28,28,28,28,28
Cyprus,4,4,4,4,4,4
East Ham,12,12,12,12,12,12
East Village,43,43,43,43,43,43
Forest Gate,12,12,12,12,12,12
Manor Park,14,14,14,14,14,14
Maryland,26,26,26,26,26,26
North Woolwich,4,4,4,4,4,4


### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(newham_venues['Venue Category'].unique())))

## 3. Analyze Each Neighborhood

#### Here we apply one hot encoding to all the venues. So now the number of columns becomes 100

In [ ]:
# one hot encoding
newham_onehot = pd.get_dummies(newham_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newham_onehot['Neighborhood'] = newham_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newham_onehot.columns[-1]] + list(newham_onehot.columns[:-1])
newham_onehot = newham_onehot[fixed_columns]

newham_onehot.head()

#### And let's examine the new dataframe size.

In [ ]:
newham_onehot.shape

In [ ]:
len((newham_onehot[newham_onehot["Shopping Plaza"] > 0]))

In [ ]:
len((newham_onehot[newham_onehot["Shopping Mall"] > 0]))

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
grouped_newham = newham_onehot.groupby('Neighborhood').mean().reset_index()
grouped_newham

### Let's print each neighborhood along with the top 10 most common venues

In [ ]:
top_venues = 10

for hood in grouped_newham['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped_newham[grouped_newham['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

### let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
sorted_neighborhoods_venues = pd.DataFrame(columns=columns)
sorted_neighborhoods_venues['Neighborhood'] = grouped_newham['Neighborhood']

for ind in np.arange(grouped_newham.shape[0]):
    sorted_neighborhoods_venues.iloc[ind, 1:] = return_most_common_venues(grouped_newham.iloc[ind, :], top_venues)

sorted_neighborhoods_venues.head()

## 4. Cluster Neighborhoods

#### Lets use k-means to cluster the neighborhood into 5 clusters

In [ ]:
# set number of clusters
kclusters = 5

grouped_newham_clustering = grouped_newham.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_newham_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
kmeans

### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# add clustering labels
sorted_neighborhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)

newham_merged = df_coord

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
newham_merged = newham_merged.join(sorted_neighborhoods_venues.set_index('Neighborhood'), on='Neighborhood')

newham_merged.head() # check the last columns!

### Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[51.5460532, 0.0059223], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newham_merged['Latitude'], newham_merged['Longitude'], newham_merged['Neighborhood'], newham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [ ]:
newham_merged.loc[newham_merged['Cluster Labels']== 0, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

In [ ]:
newham_merged.loc[newham_merged['Cluster Labels']== 1, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

In [ ]:
newham_merged.loc[newham_merged['Cluster Labels']== 2, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

In [ ]:
newham_merged.loc[newham_merged['Cluster Labels']== 3, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

In [ ]:
newham_merged.loc[newham_merged['Cluster Labels']== 4, newham_merged.columns[[0] + list(range(5, newham_merged.shape[1]))]]

### Results

### Let's examine the 5 clusters

Cluster 0 and Cluster 3 has got neighborhoods with at least one Shopping Plazza which is the same as Shopping Mall

Cluster 0: This cluster has got Shopping Plazza in Beckton neighborhood.

Cluster 3: This cluster has got a Shopping Mall in Plashet neighborhood

But cluster 1, cluster 2 and cluster 4 results shows neighborhoods without shopping plazza or shopping mall present in those clusters.

## Conclusion

I would recomend the prospective project developers to think about opening shopping malls or shopping plazzas in neighborhoods where there is non present. Another benefit of capitalizing in these neighborhoods is because their will be no competition from existing malls, if not very little.